<a href="https://rangle.io"><img src="rangle_wordmark_red.png" alt="Rangle Logo" style="width: 200px;"/></a>

# LDA on Sagemaker

This Notebook presents the simple use case of training and deploying a topic model in the cloud.

## Before You Start

Running this notebook might incure costs. Please make sure you are aware of the costs and don't forget to delete any remaining resources when you're finished.

## Who We Are

<a href="https://rangle.io">Rangle</a> accelerates digital innovation.
We at Rangle believe that Artificial Intelligence (AI) and Machine Learning (ML) have the potential to redefine what it means to build software and provide great user experience.

We work with our clients together to integrate AI into their product strategy decisions and build out novel user experiences and smart applications.

## What are Topic Models?

In natural language processing, [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a generative statistical model that allows categorizing observations (e.g. documents) into groups based on similarity.
These groups (or topics) are derived from the data and can be of interest themselves.
Topic modeling techniques such as LDA can thus be used to categorized documents and also learn about the topics present in the data.

LDA posits that each document is a mixture of a small number of topics and that each word's presence is attributable to one of the document's topics.
LDA is an unsupervised learning technique.
Therefore the topics are not specified up front, and are not guaranteed to align with how a human may naturally categorize documents.
Topics are learned as a probability distribution over the words that occur in each document. 
Each document, in turn, is described as a mixture of topics.

## Why Amazon Sagemaker?

Here we use [Amazon Sagemake](https://aws.amazon.com/sagemaker/) to both train and [deploy](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html) the model.

<img src="https://docs.aws.amazon.com/sagemaker/latest/dg/images/sagemaker-architecture.png" alt="Rangle Logo" style="width: 700px;"/>

Amazon Sagemaker has the benefit that the user does not have to implement most of the code associated with training and deployment.
The whole end-to-end process of training and deploying a model consists of 3 steps.

### Data Preparation

Data is transferred to an S3 bucket where it is easily accessible to Model Training.

### Model Training

Model Training consists of selecting a docker image containing the training code.
The user can provide their own custom training code.
Below we take advantage of the large library of avaialble machine learning algorithms and just select the 'LDA' image provided by Amazon.
The docker image then gets deployed to an [instance](https://aws.amazon.com/sagemaker/pricing/instance-types/) and trains on the data contained in the S3 bucket after which it automatically shuts down to save costs.
Selecting the right instance allows the user to trade-off training time and costs.
The training concludes by saving the trained model and associated assets to S3.

### Deployment

Deployment consists of selecting a Deployment instance and the communication protocol.
Once the model is deployed the user can use it to make predictions by sending new data to the endpoint, which will answer with the model's predictions.

## Outline

The following code will implement the above three steps.

- Setup
- Data Preparation
- Data Visualization
- AWS credentials
- Training the Model
- Evaluating the Model Output
- Making Predictions

## Setup

In [ ]:
%matplotlib inline

import os
import time
import re
import tarfile
import json
import requests

# Amazon Web Services (AWS) SDK for Python
import boto3

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

import mxnet as mx
import numpy as np

# SageMaker Python SDK
import sagemaker
from sagemaker.amazon.common import numpy_to_record_serializer
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

## Data Preparation

Here we'll use text data collected from the New York Times.
The data is already in the right form (word counts).

In [ ]:
# word counts
with open('nyt_data.txt') as f:
    word_counts = f.readlines()
# bag_of_words = [x.strip().strip('\n').strip("'") for x in bag_of_words] 
word_counts = [x.strip() for x in word_counts] 

# vocabulary (i.e. unique words)
with open('nyt_vocab.dat') as f:
    word = f.readlines()
word = np.array([x.strip().strip('\n').strip("'") for x in word])

vocabulary_size = len(word)

print(len(word_counts), "bags of words")
print(vocabulary_size, "words in vocabulary")

In [ ]:
# tranlate bag of words into one giant matrix
bags_of_words = np.zeros([len(word_counts), len(word)])

for row in range(len(word_counts)):
    for col in word_counts[row].split(','):
        bags_of_words[row, int(col.split(':')[0])-1] = int(col.split(':')[1])

If you had some pre-labeled data you'd split the data into training and testing data sets to estimate real-world accuracy.

In [ ]:
nbags_of_words = min(bags_of_words.shape[0], 10_000) # speed up testing with fewer documents
ntopics = 10

nbags_of_words_training = int(0.95*nbags_of_words)
nbags_of_words_test = nbags_of_words - nbags_of_words_training

bags_of_words_training = bags_of_words[:nbags_of_words_training]
bags_of_words_test = bags_of_words[nbags_of_words_training:nbags_of_words]

print('training set dimensions = {}'.format(bags_of_words_training.shape))
print('test set dimensions = {}'.format(bags_of_words_test.shape))

## Data Visualization

Here we visualize (part of) the matrix we created above.
Each row corresponds to a single document (or bag-of-words).
Each columns represents a unique word.
Columns are roughly sorted according to frequency.
The brightness indicates the frequency of a particular word in a particular document.

In [ ]:
sns.heatmap(bags_of_words[:500, :500], vmax = 2);

In [ ]:
# frequency of words in document 3
for i in range(10):
    print("{:10} {}".format(word[i], int(bags_of_words[3][i])))
print("...")

In [ ]:
# lenght of document 3
int(bags_of_words[3].sum())

In [ ]:
# average lenght
bags_of_words.sum(axis=1).mean()

## AWS credentials

All assets should be withing the same region as this notebook instance, training, and hosting.
More information can be found at [AWS Identity and Access Management](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_identifiers.html#identifiers-arns).

**Before you proceed:** Modify the `credentials.json` file to include the right role, bucket and prefix, e.g.:

```
{
  "role": "arn:aws:iam::123456789012:role/sagemaker-topic-modeling",
  "bucket": "companyname-topic-modeling-test",
  "prefix": "sagemaker/lda-model"
}
```

### S3 Bucket

Stores training data and model data output.

### Prefix

The location in the bucket where this notebook's input and and output data will be stored. (The default value is sufficient.)

### Role

The IAM Role ARN used to give training and hosting access to your data.

In [ ]:
credentials_filename = 'credentials.json'

with open(credentials_filename) as f:
    cred = json.load(f)

print('Training input/output will be saved to: {}/{}'.format(cred['bucket'], cred['prefix']))
print('IAM Role: {}'.format(cred['role']))

## Training the Model

Here we fit an LDA model to the corpus (as opposed to generating a corpus).
Amazon SageMaker LDA uses a spectral tensor decomposition technique to determine the LDA model parameters $(\alpha, \beta)$ which most likely describes the corpus.

![](https://upload.wikimedia.org/wikipedia/commons/4/4d/Smoothed_LDA.png)

- $\alpha$ is the parameter of the Dirichlet prior on the per-document topic distributions,
- $\beta$ is the parameter of the Dirichlet prior on the per-topic word distribution,
- $\theta_m$ is the topic distribution for document m,
- $\varphi_k$ is the word distribution for topic k,
- $z_{mn}$ is the topic for the n-th word in document m, and
- $w_{mn}$ is the specific word.

### Upload data to an S3 bucket

Here we convert the documents to MXNet RecordIO Protobuf format. 

In [ ]:
%%time
# 35 s

# convert documents_training to Protobuf RecordIO format
recordio_protobuf_serializer = numpy_to_record_serializer()
fbuffer = recordio_protobuf_serializer(bags_of_words_training)

# upload to S3 in bucket/prefix/train
fname = 'lda.data'
s3_object = os.path.join(cred['prefix'], 'train', fname)
boto3.Session().resource('s3').Bucket(cred['bucket']).\
    Object(s3_object).upload_fileobj(fbuffer)

s3_train_data = 's3://{}/{}'.format(cred['bucket'], s3_object)
print('Uploaded data to S3: {}'.format(s3_train_data))

### Create LDA Container

In [ ]:
# SageMaker LDA Docker container
region_name = boto3.Session().region_name
container = get_image_uri(boto3.Session().region_name, 'lda')

print('Using SageMaker LDA container: {} ({})'.format(container, region_name))

### Initalize Session

In [ ]:
session = sagemaker.Session()

# specify general training job information
lda = sagemaker.estimator.Estimator(
    container,
    cred['role'],
    output_path = 's3://{}/{}/output'.format(cred['bucket'], cred['prefix']),
    train_instance_count = 1,
    train_instance_type = 'ml.c4.2xlarge',
    sagemaker_session = session,
)

# set algorithm-specific hyperparameters
lda.set_hyperparameters(
    num_topics = ntopics,
    feature_dim = vocabulary_size,
    mini_batch_size = nbags_of_words_training,
    alpha0 = 1.0,
)

### Training

After executing the code in the following cell login at:
[https://console.aws.amazon.com/console/home](https://console.aws.amazon.com/console/home)

And follow the training progress at: `Amazon SageMaker > Training jobs`.
(You might have to change the region in the top-right to: `US East`.)

In [ ]:
# 3:20 min

# run the training job on input data stored in S3
start = time.time()
try:
    lda.fit({'train': s3_train_data})
except RuntimeError as e:
    print(e)  

end = time.time()
print("Training took", end - start, "seconds")

In [ ]:
print('Training job name: {}'.format(lda.latest_training_job.job_name))

## Evaluating the Model Output

Here we download the model assets from Amzon S3 and inspect the model.
The model basically constist of two arrays containing the $\alpha$ and $\beta$ parameters that were estimated during training.

In [ ]:
# download and extract the model file from S3
job_name = lda.latest_training_job.job_name
model_fname = 'model.tar.gz'
model_object = os.path.join(cred['prefix'], 'output', job_name, 'output', model_fname)
boto3.Session().resource('s3').Bucket(cred['bucket']).Object(model_object).download_file(fname)
with tarfile.open(fname) as tar:
    tar.extractall()
print('Downloaded and extracted model tarball: {}'.format(model_object))

# obtain the model file
model_list = [fname for fname in os.listdir('.') if fname.startswith('model_')]
model_fname = model_list[0]
print('Found model file: {}'.format(model_fname))

# get the model from the model file and store in Numpy arrays
alpha, beta = mx.ndarray.load(model_fname)
learned_alpha = alpha.asnumpy()
learned_beta = beta.asnumpy()

print('\nLearned alpha.shape = {}'.format(learned_alpha.shape))
print('Learned beta.shape = {}'.format(learned_beta.shape))

In [ ]:
# visualize alpha
sns.lineplot(range(len(learned_alpha)), learned_alpha);

In [ ]:
# visualize beta
sns.heatmap(learned_beta, vmax=0.01); # (topics, words)

In [ ]:
# print most important words for a given topic
topic_nr = 1

beta = learned_beta[topic_nr]
idx = np.argsort(beta)

print("Topic", topic_nr)
print("=====================")
for i in idx[:-11:-1]:
    print("{:12} {:f}".format(word[i], beta[i]))

#### Interpretation

When restricting the number of topics to 10 (`ntopics = 10`), we get the following result for topic 1.

```
Topic 1
=====================
music        0.006595
play         0.005841
life         0.003958
performance  0.003903
man          0.003707
write        0.003601
young        0.003494
art          0.003459
program      0.003393
offer        0.003372
```

Intutitively this seems to correspond to the arts section of a newspaper.
Change the `topic_nr` variable above and explore the other 9 topics.
What happens if you re-run the training after chaning the `ntopics` variable?
Does increasing or decreasing the topic number improve the quality of the topics?

## Prediction

Here we predict the topic mixture representing a given document.

We create an inference endpoint using the SageMaker Python SDK deploy() function from the job we defined above. We specify the instance type where inference is computed as well as an initial number of instances to spin up.

In [ ]:
# 8 min

lda_inference = lda.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m4.xlarge',
)

In [ ]:
print('Endpoint name: {}'.format(lda_inference.endpoint))

In [ ]:
# configure data format (CSV, JSON, RECORDIO Protobu)
lda_inference.content_type = 'text/csv'
lda_inference.serializer = csv_serializer
lda_inference.deserializer = json_deserializer

In [ ]:
# query endpoint
results = lda_inference.predict(bags_of_words_test[:3])

import json
print(json.dumps(results, sort_keys=True, indent=2))

In [ ]:
# let's predict on the whole test set
results = lda_inference.predict(bags_of_words_test)

In [ ]:
len(results['predictions'])

In [ ]:
# delete endpoint
sagemaker.Session().delete_endpoint(lda_inference.endpoint)